In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Set to None to display all rows
import json
import requests
import pickle
import urllib
import psycopg2
from psycopg2.extras import execute_values
from datetime import datetime, timedelta, timezone
import os

BASE_URL = 'https://api.prop-odds.com'
API_KEY = '5DQv4UzUztm6itoSLRaFdXDi5Dt4zGNFT1DvEFh0D0M'
# API_KEY = 'EEfdxYtFG2BgeCA2xDfM8KMgHvZVbYEaDGS5lCd38U'
# API_KEY = '9WnS02ST9fLkXsMZgFQKBJdzGwdtIt5yx0s7YFrHfc'

In [2]:
def get_request(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()

    print('Request failed with status:', response.status_code)
    return {}

In [6]:
def get_db_connection(db_prefix):
    """
    Establish a connection to the specified database using environment variables.

    Args:
        db_prefix (str): The prefix for the database environment variables (e.g., 'NHL_DB_', 'PROP_ODDS_DB_').

    Returns:
        tuple: A tuple containing the database connection and cursor.
    """
    db_host = os.getenv(f'{db_prefix}HOST')
    db_port = os.getenv(f'{db_prefix}PORT')
    db_name = os.getenv(f'{db_prefix}NAME')
    db_user = os.getenv(f'{db_prefix}USER')
    db_password = os.getenv(f'{db_prefix}PASSWORD')

    try:
        conn = psycopg2.connect(
            host=db_host,
            port=db_port,
            dbname=db_name,
            user=db_user,
            password=db_password
        )
        cursor = conn.cursor()
        return conn, cursor
    except Exception as e:
        print(f"Failed to connect to the {db_prefix[:-1]} database:", e)
        return None, None

In [3]:
def fetch_and_store_nhl_games(query_date=None):
    """
    Fetch Prop Odds games for a given date and store them in the PostgreSQL prop_odds database.

    Args:
        query_date (str, optional): The date to query in 'YYYY-MM-DD' format. Defaults to today.

    Returns:
        list: A list of game dictionaries retrieved from the API.
    """
    if query_date is None:
        query_date = datetime.now()
    else:
        query_date = datetime.strptime(query_date, '%Y-%m-%d')

    query_params = {
        'date': query_date.strftime('%Y-%m-%d'),
        'tz': 'America/New_York',
        'api_key': API_KEY,
    }
    params = urllib.parse.urlencode(query_params)
    url = f"{BASE_URL}/beta/games/nhl?{params}"  # Adjust the endpoint as needed
    games_data = get_request(url)

    if not games_data:
        print("No data retrieved from API.")
        return

    games = games_data.get('games', [])

    # Establish a connection using the helper function
    conn, cursor = get_db_connection('PROP_ODDS_DB_')

    if not conn or not cursor:
        return

    try:
        # Define the insert query
        insert_query = """
            INSERT INTO game_info (
                id, 
                game_id, 
                away_team, 
                home_team, 
                start_timestamp
            )
            VALUES %s
            ON CONFLICT (id) DO UPDATE
            SET 
                game_id = EXCLUDED.game_id,
                away_team = EXCLUDED.away_team,
                home_team = EXCLUDED.home_team,
                start_timestamp = EXCLUDED.start_timestamp,
                last_updated = CURRENT_TIMESTAMP;
        """

        # Prepare data for insertion
        records_to_insert = [
            (
                game['id'],
                game['game_id'],
                game['away_team'],
                game['home_team'],
                game['start_timestamp']
            )
            for game in games
        ]

        # Use execute_values for efficient bulk insertion
        execute_values(cursor, insert_query, records_to_insert)

        # Commit the transaction
        conn.commit()
        print(f"Inserted/Updated {len(records_to_insert)} records into prop_odds.game_info table.")

    except Exception as e:
        print("An error occurred while inserting data into the Prop Odds database:", e)
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

    return games

In [4]:
def get_today_date():
    # Get the current UTC time
    utc_dt = datetime.now(timezone.utc)

    # Calculate the offset for CST (UTC-6)
    cst_offset = timedelta(hours=-6)

    # Apply the offset to get CST time
    cst_dt = utc_dt + cst_offset

    # Format the date as 'yyyy-mm-dd'
    return cst_dt.strftime('%Y-%m-%d')

# Get the CST date
cst_date = get_today_date()
print(cst_date)

2024-12-11


In [7]:
fetch_and_store_nhl_games('2024-12-08')

Inserted/Updated 7 records into prop_odds.game_info table.


[{'id': 401688035,
  'game_id': 'b542cf3faf8c2e34776f052f45c2dce5',
  'away_team': 'Seattle Kraken',
  'home_team': 'New York Rangers',
  'start_timestamp': '2024-12-08T18:00:00Z',
  'participants': [],
  'is_started': None},
 {'id': 401688036,
  'game_id': '03cf6c6c50ba700843e527343e61371f',
  'away_team': 'Tampa Bay Lightning',
  'home_team': 'Vancouver Canucks',
  'start_timestamp': '2024-12-08T21:00:00Z',
  'participants': [],
  'is_started': None},
 {'id': 401688037,
  'game_id': '33400697484eb2f395d2324e35219c31',
  'away_team': 'New York Islanders',
  'home_team': 'Ottawa Senators',
  'start_timestamp': '2024-12-08T22:00:00Z',
  'participants': [],
  'is_started': None},
 {'id': 401688038,
  'game_id': '2e487e35039b8eb1398fe9a709651758',
  'away_team': 'Columbus Blue Jackets',
  'home_team': 'Winnipeg Jets',
  'start_timestamp': '2024-12-08T23:00:00Z',
  'participants': [],
  'is_started': None},
 {'id': 401688039,
  'game_id': 'c1fa7d3f30fdb408b78917509d1633c3',
  'away_team': 

In [11]:
def get_nhl_games_from_db(query_date=None):
    """
    Retrieve NHL games from the PostgreSQL prop_odds database for a given date.

    Args:
        query_date (str, optional): The date to query in 'YYYY-MM-DD' format. Defaults to today.

    Returns:
        list: A list of game dictionaries retrieved from the database.
    """
    if query_date is None:
        query_date = get_today_date()
    else:
        query_date = datetime.strptime(query_date, '%Y-%m-%d')
    
    try:
        conn, cursor = get_db_connection('PROP_ODDS_DB_')
        if not conn or not cursor:
            return []
        
        query = """
            SELECT id, game_id, away_team, home_team, start_timestamp
            FROM game_info
            WHERE start_timestamp::date = %s;
        """
        cursor.execute(query, (query_date.strftime('%Y-%m-%d'),))
        rows = cursor.fetchall()
        
        games = []
        for row in rows:
            game = {
                'id': row[0],
                'game_id': row[1],
                'away_team': row[2],
                'home_team': row[3],
                'start_timestamp': row[4]
            }
            games.append(game)
        
        return games
    
    except Exception as e:
        print("An error occurred while reading data from the database:", e)
        return []
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# get_nhl_games_from_db('2024-12-11')

In [12]:
def fetch_game_markets(game_id, market_name=None):
    """
    Fetch game markets for a given game ID and optionally a specific market name, then store them in the database.

    Args:
        game_id (str): The unique identifier for the game.
        market_name (str, optional): The specific market to fetch. Defaults to None, which fetches all markets.

    Returns:
        dict: The market data retrieved from the API.
    """
    if market_name is not None:
        url = f"{BASE_URL}/beta/odds/{game_id}/{market_name}?api_key={API_KEY}"
    else:
        url = f"{BASE_URL}/beta/markets/{game_id}?api_key={API_KEY}"

    try:
        # Make an HTTP GET request
        response = requests.get(url)

        # Check for a successful response
        if response.status_code == 200:
            return response.json()  # Assuming the response is in JSON format
        else:
            print(f"Request failed with status code {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return None
# fetch_game_markets('c1fa7d3f30fdb408b78917509d1633c3')

In [9]:
# game_id = "c1fa7d3f30fdb408b78917509d1633c3"
# data = fetch_game_markets(game_id, 'player_shots_over_under' )
# data
# for sportsbook in data['sportsbooks']:
#     print(sportsbook)
# # Create empty dict to store outcomes by bookie
# outcomes_by_bookie = {}

# for sportsbook in data['sportsbooks']:
#     bookie_key = sportsbook['bookie_key']
#     outcomes_by_bookie[bookie_key] = []
    
#     # Get all outcomes for this sportsbook
#     for outcome in sportsbook['market']['outcomes']:
#         outcomes_by_bookie[bookie_key].append({
#             'timestamp': outcome['timestamp'],
#             'handicap': outcome['handicap'], 
#             'odds': outcome['odds'],
#             'participant_name': outcome['participant_name'],
#             'name': outcome['name'],
#             'description': outcome['description']
#         })

In [13]:
def format_player_name(name):
    """
    Extracts the player's name and the bet type from the outcome name.

    Args:
        name (str): The name string from the outcome (e.g., "Artturi Lehkonen Over 0.5").

    Returns:
        tuple: A tuple containing the player's name and the bet type ('Over' or 'Under').
    """
    parts = name.split()

    # Ensure the name has at least three parts: Player Name, Bet Type, Handicap
    if len(parts) < 3:
        return None, None

    # Extract bet type ('Over' or 'Under')
    bet_type = parts[-2]
    if bet_type not in ['Over', 'Under']:
        return None, None

    # Extract player name by joining all parts except the last two
    player_name = ' '.join(parts[:-2])
    return player_name, bet_type

In [14]:
def insert_outcome_into_db(outcome):
    """
    Inserts a single outcome record into the Outcomes table.
    If the record already exists, it does nothing.
    
    Parameters:
        outcome (dict): The outcome data to insert.
    """
    try:
        # Establish a connection using the helper function
        conn, cursor = get_db_connection('PROP_ODDS_DB_')

        if not conn or not cursor:
            return
        
        insert_query = """
        INSERT INTO player_shots_ou (
            game_id,
            sportsbook,
            player,
            ou,
            handicap,
            odds,
            timestamp
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (game_id, sportsbook, player, ou, handicap, odds, timestamp) DO NOTHING
        """
        
        data_tuple = (
            outcome['game_id'],
            outcome['sportsbook'],
            outcome['player'],
            outcome['ou'],
            outcome['handicap'],
            outcome['odds'],
            outcome['timestamp']
        )
        
        cursor.execute(insert_query, data_tuple)
        conn.commit()
        
    except Exception as e:
        print("An error occurred while inserting data into the database:", e)
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [16]:
def process_game_markets(game_id, data):
    """
    Processes game markets data and inserts outcomes into the database.
    
    Parameters:
        game_id (str): The unique identifier for the game.
        data (dict): The data returned from `get_game_markets`.
    """
    supported_bookies = ['fanduel', 'pinnacle', 'draftkings', 'betmgm','barstool',
                        'betrivers','pointsbet', 'fliff', 'hardrock', 'betonline', 'fanatics']
    
    for sportsbook in data['sportsbooks']:
        bookie_key = sportsbook['bookie_key']
        market = sportsbook['market']
        market_key = market['market_key']
        
        if bookie_key not in supported_bookies:
            print(f"Unsupported bookmaker: {bookie_key}")
            continue  # Skip unsupported bookmakers
        
        for outcome in market['outcomes']:
            timestamp = outcome['timestamp']
            handicap = outcome.get('handicap')
            odds = outcome.get('odds')
            participant = outcome.get('participant')
            participant_name = outcome.get('participant_name')
            name = outcome.get('name')
            description = outcome.get('description')
            
            player_name, over_under = format_player_name(name)
            
            if not player_name or not over_under:
                print(f"Invalid outcome name format: {name}")
                continue  # Skip if the name format is incorrect
            
            # Create a dictionary for the outcome
            outcome_data = {
                'game_id': game_id,
                'sportsbook': bookie_key,
                'player': player_name,
                'ou': over_under,
                'handicap': handicap,
                'odds': odds,
                'timestamp': timestamp,
            }
            
            # Pass the dictionary to the insert function
            insert_outcome_into_db(outcome_data)
# game_id = "c1fa7d3f30fdb408b78917509d1633c3"
# data = fetch_game_markets(game_id, 'player_shots_over_under' )
# process_game_markets(game_id, data)

In [17]:
def process_nhl_games_for_date(date, market='player_shots_over_under'):
    games = get_nhl_games_from_db(date)
    if not games:
        fetch_and_store_nhl_games(date)
        games = get_nhl_games_from_db(date)
    game_ids = [game['game_id'] for game in games]
    for game_id in game_ids:
        market_data = fetch_game_markets(game_id, market)
        process_game_markets(game_id, market_data)

In [25]:
process_nhl_games_for_date('2024-12-10')

Inserted/Updated 11 records into prop_odds.game_info table.
Unsupported bookmaker: betopenly
Unsupported bookmaker: betopenly
Unsupported bookmaker: betopenly


In [23]:
def get_player_shots_ou_odds(player_name=None, team_abbreviation=None, query_date=None):
    """
    Retrieve player shot over/under odds from the PostgreSQL prop_odds database
    based on a specific player name or team abbreviation for a given date.

    Args:
        player_name (str, optional): The full name of the player to filter odds by.
        team_abbreviation (str, optional): The abbreviation of the team to filter odds by.
        query_date (str, optional): The date to query in 'YYYY-MM-DD' format. Defaults to today.

    Returns:
        list: A list of dictionaries containing player shot OU odds.
    """
    from datetime import datetime
    import psycopg2

    # Validate that at least one filter is provided
    if not player_name and not team_abbreviation:
        print("Either player_name or team_abbreviation must be provided.")
        return []

    # Determine the query date
    if query_date is None:
        query_date = get_today_date()
    else:
        try:
            query_date = datetime.strptime(query_date, '%Y-%m-%d')
        except ValueError:
            print("Invalid date format. Please use 'YYYY-MM-DD'.")
            return []

    try:
        # Establish a database connection
        conn, cursor = get_db_connection('PROP_ODDS_DB_')
        if not conn or not cursor:
            print("Database connection failed.")
            return []

        # Initialize the base query
        base_query = """
            SELECT 
                pso.game_id,
                pso.sportsbook,
                pso.player,
                pso.ou,
                pso.handicap,
                pso.odds,
                pso.timestamp
            FROM player_shots_ou pso
            JOIN game_info gi ON pso.game_id = gi.game_id
            WHERE gi.start_timestamp::date = %s
        """

        # Initialize parameters list with the query date
        params = [query_date.strftime('%Y-%m-%d')]

        # Add filters based on provided arguments
        if player_name and team_abbreviation:
            # If both player and team filters are provided
            base_query += """
                AND pso.player ILIKE %s
                AND (gi.away_team ILIKE %s OR gi.home_team ILIKE %s)
            """
            params.extend([player_name, team_abbreviation, team_abbreviation])
        elif player_name:
            # If only player filter is provided
            base_query += " AND pso.player ILIKE %s"
            params.append(player_name)
        else:
            # If only team filter is provided
            base_query += " AND (gi.away_team ILIKE %s OR gi.home_team ILIKE %s)"
            params.extend([team_abbreviation, team_abbreviation])

        # Debugging: Print the final SQL query with parameters
        print("Executing SQL Query:")
        print(cursor.mogrify(base_query, tuple(params)).decode('utf-8'))

        # Execute the query with parameters
        cursor.execute(base_query, tuple(params))
        rows = cursor.fetchall()

        # Debugging: Print the number of rows fetched
        print(f"Number of rows fetched: {len(rows)}")

        # Process the retrieved rows
        odds_list = []
        for row in rows:
            odds = {
                'game_id': row[0],
                'sportsbook': row[1],
                'player': row[2],
                'ou': row[3],
                'handicap': row[4],
                'odds': row[5],
                'timestamp': row[6]
            }
            odds_list.append(odds)

        return odds_list

    except psycopg2.Error as e:
        print("A database error occurred:", e)
        return []
    except Exception as e:
        print("An unexpected error occurred:", e)
        return []
    finally:
        # Ensure the cursor and connection are closed
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [27]:
odds = get_player_shots_ou_odds(player_name='Artturi Lehkonen', query_date='2024-12-10')
odds

Executing SQL Query:

            SELECT 
                pso.game_id,
                pso.sportsbook,
                pso.player,
                pso.ou,
                pso.handicap,
                pso.odds,
                pso.timestamp
            FROM player_shots_ou pso
            JOIN game_info gi ON pso.game_id = gi.game_id
            WHERE gi.start_timestamp::date = '2024-12-10'
         AND pso.player ILIKE 'Artturi Lehkonen'
Number of rows fetched: 39


[{'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'barstool',
  'player': 'Artturi Lehkonen',
  'ou': 'Over',
  'handicap': 1.5,
  'odds': -320,
  'timestamp': datetime.datetime(2024, 12, 11, 0, 8, 42, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))},
 {'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'barstool',
  'player': 'Artturi Lehkonen',
  'ou': 'Over',
  'handicap': 2.5,
  'odds': 100,
  'timestamp': datetime.datetime(2024, 12, 11, 0, 8, 42, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))},
 {'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'barstool',
  'player': 'Artturi Lehkonen',
  'ou': 'Over',
  'handicap': 3.5,
  'odds': 250,
  'timestamp': datetime.datetime(2024, 12, 11, 0, 8, 42, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=64800)))},
 {'game_id': 'ba8a3041639b65d6d085fb72d73b7b98',
  'sportsbook': 'barstool',
  'player': 'Artturi Lehkonen',
  'ou': 'Under',
  'handic